# GAIN

In [ ]:
import os
import sys
from six.moves import cPickle
import scipy.ndimage as nd
import numpy as np
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib import colors as mpl_colors
from GAIN import GAIN
from SEC import SEC
from dataset import dataset

%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

class Inference_GAIN(GAIN):
    def inference(self, batch_size, gpu_frac, eps=1e-5):
        self.cmap = mpl_colors.LinearSegmentedColormap.from_list('Custom cmap', [
            (0.0, 0.0, 0.0), (0.5, 0.0, 0.0), (0.0, 0.5, 0.0), (0.5, 0.5, 0.0),
            (0.0, 0.0, 0.5), (0.5, 0.0, 0.5), (0.0, 0.5, 0.5), (0.5, 0.5, 0.5),
            (0.25, 0.0, 0.0), (0.75, 0.0, 0.0), (0.25, 0.5, 0.0), (0.75, 0.5, 0.0),
            (0.25, 0.0, 0.5), (0.75, 0.0, 0.5), (0.25, 0.5, 0.5), (0.75, 0.5, 0.5),
            (0.0, 0.25, 0.0), (0.5, 0.25, 0.0), (0.0, 0.75, 0.0), (0.5, 0.75, 0.0),
            (0.0, 0.25, 0.5)], 21)
        gpu_options = tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=gpu_frac))
        self.sess = tf.Session(config=gpu_options)
        x, gt, _, _, id_of_image, iterator_train = self.data.next_batch(category="train",batch_size=batch_size,epoches=-1)
        self.build()
        self.saver["norm"] = tf.train.Saver(max_to_keep=2,var_list=self.trainable_list)
        with self.sess.as_default():
            self.sess.run(tf.global_variables_initializer())
            self.sess.run(tf.local_variables_initializer())
            self.sess.run(iterator_train.initializer)
            if self.config.get("model_path",False) is not False: self.restore_from_model(self.saver["norm"], self.config.get("model_path"), checkpoint=False)
            epoch, i, iterations_per_epoch_train = 0.0, 0, self.data.get_data_len()//batch_size
            while epoch < 1:
                data_x, data_gt, img_id = self.sess.run([x, gt, id_of_image])
                params = {self.net["input"]:data_x, self.net["drop_prob"]:0.5}
                preds = self.sess.run(self.net["fc8-softmax"], feed_dict=params)
                for pred in preds:
                    cimg_id = img_id[0].decode("utf-8")
                    if not os.path.exists('sec-preds/{}.png'.format(cimg_id)):
                        continue
                    img = Image.open("data/VOCdevkit/VOC2012/JPEGImages/{}.jpg".format(cimg_id)).resize((321,321), Image.ANTIALIAS)
                    scores_exp = np.exp(pred-np.max(pred, axis=2, keepdims=True))
                    probs = scores_exp/np.sum(scores_exp, axis=2, keepdims=True)
                    probs = nd.zoom(probs, (321/probs.shape[0], 321/probs.shape[1], 1.0), order=1)
                    probs[probs<eps] = eps
                    mask = np.argmax(probs, axis=2)
                    
                    fig = plt.figure(figsize=(20,20))
                    ax1, ax2, ax3, ax4 = fig.add_subplot('141'), fig.add_subplot('142'), fig.add_subplot('143'), fig.add_subplot('144')
                    ax1.imshow(img)
                    ax2.imshow(cPickle.load(open("sec-preds/{}.png".format(cimg_id), 'rb')), , vmin=0, vmax=21, cmap=self.cmap)
                    ax3.matshow(mask, vmin=0, vmax=21, cmap=self.cmap)
                    ax4.imshow(data_gt.reshape((321,321)), cmap='gray')
                    ax1.set_title("image")
                    ax2.set_title("sec")
                    ax3.set_title("gain")
                    ax4.set_title("gt_mask")
                    ax1.set_axis_off()
                    ax2.set_axis_off()
                    ax3.set_axis_off()
                    ax4.set_axis_off()
                    plt.show()
                i+=1
                epoch = i/iterations_per_epoch_train


os.environ["CUDA_VISIBLE_DEVICES"] = '0'
ckpt_path = "gain-saver/norm-2999"
gain = Inference_GAIN({"data":dataset({"batch_size":1, "input_size":(321,321), "epoches":30, "category_num":21, "categorys":["train"]}), "batch_size":1, "input_size":(321,321), "epoches":30, "category_num":21, "model_path":ckpt_path, "accum_num":16})
gain.inference(1,0.05)

# SEC Baseline

In [ ]:
class Inference_SEC(SEC):
    def inference(self, batch_size, gpu_frac, eps=1e-5):
        self.cmap = mpl_colors.LinearSegmentedColormap.from_list('Custom cmap', [
            (0.0, 0.0, 0.0), (0.5, 0.0, 0.0), (0.0, 0.5, 0.0), (0.5, 0.5, 0.0),
            (0.0, 0.0, 0.5), (0.5, 0.0, 0.5), (0.0, 0.5, 0.5), (0.5, 0.5, 0.5),
            (0.25, 0.0, 0.0), (0.75, 0.0, 0.0), (0.25, 0.5, 0.0), (0.75, 0.5, 0.0),
            (0.25, 0.0, 0.5), (0.75, 0.0, 0.5), (0.25, 0.5, 0.5), (0.75, 0.5, 0.5),
            (0.0, 0.25, 0.0), (0.5, 0.25, 0.0), (0.0, 0.75, 0.0), (0.5, 0.75, 0.0),
            (0.0, 0.25, 0.5)], 21)
        gpu_options = tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=gpu_frac))
        self.sess = tf.Session(config=gpu_options)
        x, gt, _, _, id_of_image, iterator_train = self.data.next_batch(category="train",batch_size=batch_size,epoches=-1)
        self.build()
        self.saver["norm"] = tf.train.Saver(max_to_keep=2,var_list=self.trainable_list)
        with self.sess.as_default():
            self.sess.run(tf.global_variables_initializer())
            self.sess.run(tf.local_variables_initializer())
            self.sess.run(iterator_train.initializer)
            if self.config.get("model_path",False) is not False: self.restore_from_model(self.saver["norm"], self.config.get("model_path"), checkpoint=False)
            epoch, i, iterations_per_epoch_train = 0.0, 0, self.data.get_data_len()//batch_size
            while epoch < 1:
                data_x, data_gt, img_id = self.sess.run([x, gt, id_of_image])
                params = {self.net["input"]:data_x, self.net["drop_prob"]:0.5}
                preds = self.sess.run(self.net["fc8-softmax"], feed_dict=params)
                for pred in preds:
                    img = Image.open("data/VOCdevkit/VOC2012/JPEGImages/{}.jpg".format(img_id[0].decode("utf-8"))).resize((321,321), Image.ANTIALIAS)
                    scores_exp = np.exp(pred-np.max(pred, axis=2, keepdims=True))
                    probs = scores_exp/np.sum(scores_exp, axis=2, keepdims=True)
                    probs = nd.zoom(probs, (321/probs.shape[0], 321/probs.shape[1], 1.0), order=1)
                    probs[probs<eps] = eps
                    mask = np.argmax(probs, axis=2)
                    cPickle.dump(mask, open('sec-preds/{}.pkl'.format(img_id[0].decode("utf-8")), 'wb'))
                i+=1
                epoch = i/iterations_per_epoch_train

"""
ckpt_path = "sec-saver/norm-56999"
sec = Inference_SEC({"data":dataset({"batch_size":1, "input_size":(321,321), "epoches":30, "category_num":21, "categorys":["train"]}), "batch_size":1, "input_size":(321,321), "epoches":30, "category_num":21, "model_path":ckpt_path, "accum_num":16})
sec.inference(1,0.05)
"""

len:{'train': 10582}
INFO:tensorflow:Restoring parameters from sec-saver/norm-56999
